In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()
# chat = ChatAnthropic()

b = chat.predict("How many planets are there?")

b

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)